In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

# To display figs, run in terminal: jupyter nbextension enable --py gmaps
# Open jupyter notebook from Python Data environment

In [2]:
# 2) Import the CSV from part one as a new DataFrame.
city_data_df = pd.read_csv("data/WeatherPy_challenge.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches)
0,0,Toamasina,MG,-18.17,49.38,73.40,94,75,4.70,light rain,0.0,0.0
1,1,Punta Arenas,CL,-53.15,-70.92,42.80,75,40,11.41,scattered clouds,0.0,0.0
2,2,Hobart,AU,-42.88,147.33,54.00,62,90,8.05,overcast clouds,0.0,0.0
3,3,Clyde River,CA,70.47,-68.59,24.80,85,75,3.36,broken clouds,0.0,0.0
4,4,Puteyets,RU,65.17,57.09,50.47,54,100,10.67,overcast clouds,0.0,0.0


In [3]:
# 3) Filter the DataFrame for minimum and maximum temperature preferences, and if 
# the rain or snow accumulation is 0 inches or not using conditional statements. 
# Promt the user for preferences.

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = input("Do you want it to be raining? (yes/no) ")
snowing = input("Do you want it to be snowing? (yes/no) ")


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) no


In [4]:
# Filter the dataset to find the cities that fit the criteria.

if raining == "yes":
    raining = True
else:
    raining = False
    
if snowing == "yes":
    snowing = True
else:
    snowing = False 


preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (Inches)"].astype("bool")==raining) & \
                                       (city_data_df["Snow (Inches)"].astype("bool")==snowing)
                                      ]
    
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches)
10,10,Hithadhoo,MV,-0.60,73.08,83.91,80,100,16.04,light rain,0.36,0.0
34,34,Albany,US,42.60,-73.97,84.00,63,79,9.51,light rain,0.21,0.0
50,50,Alappuzha,IN,9.49,76.33,82.98,80,94,4.94,light rain,0.83,0.0
60,60,Nautla,MX,20.22,-96.78,84.83,69,46,8.93,moderate rain,1.19,0.0
69,69,Mahina,PF,-17.51,-149.49,77.00,83,51,3.36,light rain,0.37,0.0
98,98,Lastoursville,GA,-0.81,12.73,79.74,87,100,1.25,light rain,0.70,0.0
104,104,Tabou,CI,4.42,-7.35,82.94,78,93,11.18,light rain,0.30,0.0
108,108,Kampong Chhnang,KH,12.17,104.55,79.70,85,100,3.29,light rain,0.36,0.0
125,125,Hambantota,LK,6.12,81.12,81.46,82,98,15.82,moderate rain,1.02,0.0
134,134,Bengkulu,ID,-3.80,102.27,78.75,89,100,4.43,light rain,0.16,0.0


In [5]:
city_data_df["Rain (Inches)"].astype("bool")

0      False
1      False
2      False
3      False
4      False
       ...  
547    False
548    False
549    False
550     True
551    False
Name: Rain (Inches), Length: 552, dtype: bool

In [6]:
# 4) Add the cities to a marker layer map with a pop-up marker for each city that includes:
# Hotel Name, City, County, Weather Description and Max Temp.

# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Hithadhoo,MV,83.91,light rain,-0.60,73.08,
34,Albany,US,84.00,light rain,42.60,-73.97,
50,Alappuzha,IN,82.98,light rain,9.49,76.33,
60,Nautla,MX,84.83,moderate rain,20.22,-96.78,
69,Mahina,PF,77.00,light rain,-17.51,-149.49,
98,Lastoursville,GA,79.74,light rain,-0.81,12.73,
104,Tabou,CI,82.94,light rain,4.42,-7.35,
108,Kampong Chhnang,KH,79.70,light rain,12.17,104.55,
125,Hambantota,LK,81.46,moderate rain,6.12,81.12,
134,Bengkulu,ID,78.75,light rain,-3.80,102.27,


In [7]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [8]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Hithadhoo,MV,83.91,light rain,-0.60,73.08,Scoop Guest House
34,Albany,US,84.00,light rain,42.60,-73.97,
50,Alappuzha,IN,82.98,light rain,9.49,76.33,Ramada by Wyndham Alleppey
60,Nautla,MX,84.83,moderate rain,20.22,-96.78,Casa Blanca
69,Mahina,PF,77.00,light rain,-17.51,-149.49,Guest Beach Bungalow Tahiti
98,Lastoursville,GA,79.74,light rain,-0.81,12.73,Auberge Bon Coin 02
104,Tabou,CI,82.94,light rain,4.42,-7.35,hotêl doufoulougou
108,Kampong Chhnang,KH,79.70,light rain,12.17,104.55,រមណីយដ្ឋាន ទឹកធ្លាក់ស្រែអំពិល
125,Hambantota,LK,81.46,moderate rain,6.12,81.12,Bungalow 63
134,Bengkulu,ID,78.75,light rain,-3.80,102.27,Grage Hotel Bengkulu


In [10]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [16]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
## Configure maps to use api_key
gmaps.configure(api_key=g_key)


locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# 5) Save and upload the new DataFrame as WeatherPy_vacation.csv.
# 6) Save and upload the new marker layer map as WeatherPy_vacation_map.png.

output_data_file = "data/WeatherPy_vacation.csv"
hotel_df.to_csv(output_data_file)